In [1]:
from pyspark.sql import SparkSession

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
import pandas as pd
data = pd.read_csv('/Users/sarahsha/Downloads/yelp_dataset_dataset_business_clean_withoriginalfeature.csv')

In [3]:
data.head(10)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Smoking_yes,Alcohol_beer_and_wine,Alcohol_full_bar,Alcohol_no,business_id_encoded,name_encoded,address_encoded,city_encoded,state_encoded,postal_code_encoded
0,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,...,0,0,0,1,94971,81906,98814,5,13,865
1,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,0,0,0,1,43860,73621,102520,802,16,260
2,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,...,0,0,0,1,94366,59703,2428,397,16,130
3,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,...,0,0,0,1,25032,72302,82344,27,18,621
4,n_0UpQx1hsNbnPUSlodU8w,Famous Footwear,"8522 Eager Road, Dierbergs Brentwood Point",Brentwood,MO,63144,38.627695,-90.340465,2.5,13,...,0,0,0,1,96341,27876,98191,105,13,885
5,qkRM_2X51Yqxk3btlwAQIg,Temple Beth-El,400 Pasadena Ave S,St. Petersburg,FL,33707,27.766590,-82.732983,3.5,5,...,0,0,0,1,102355,77780,61758,1035,5,527
6,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,...,0,0,0,1,73039,72302,40157,690,18,666
7,UJsufbvfyfONHeWdvAHKjA,Marshalls,21705 Village Lakes Sc Dr,Land O' Lakes,FL,34639,28.190459,-82.457380,3.5,6,...,0,0,0,1,58531,49207,37386,522,5,592
8,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,...,0,0,0,1,78721,86755,0,1082,5,477
9,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,IN,46227,39.637133,-86.127217,2.5,28,...,0,0,0,1,87355,22451,99731,472,9,747


In [4]:
spark = SparkSession.builder.appName("RecommendationSystem").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/05 22:16:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/05 22:16:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/12/05 22:16:58 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
business = spark.createDataFrame(data)

/Users/sarahsha/opt/anaconda3/envs/pyspark_env/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:479: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:
23/12/05 22:17:10 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [6]:
# define the function to provide the recommendation based on the users' input
from pyspark.sql.functions import col
def recommendations(state,city,category,day,kids=False):
    recommendation = business[['name','stars','review_count','address']+
                              ['city','state',category,day,'GoodForKids']].filter(
        (col('state')==state)&        
        (col('city')=='Santa Barbara')&
        (col(category)==True)&
        (col(day)==True)&
        (business.review_count>100)&
        (business.stars>=4.5)).orderBy(col('review_count'), ascending=False)
    if kids == 'Yes':
        recommendation = recommendation.filter(col('GoodForKids')==True)
    return recommendation[['name','stars','review_count','address']].show()

In [8]:
# collect the information from the 
def New_User_Recommendation():
    state = input(f'please enter state:')
    city = input(f'please enter city:')
    print(f"top_categoryz:'Restaurants','Food','Shopping','Beauty & Spas','Home Services','Health & Medical','Automotive','Nightlife','Local Services','Bars','Event Planning & Services','American (Traditional)','Active Life'")
    category = input(f"Tell me the category you're interested in:")
    day = input(f'please enter day:')
    kids = input(f'If will bring kids with you:')
    return recommendations(state,city,category,day,kids)

In [9]:
New_User_Recommendation()

please enter state:CA
please enter city:Santa Barbara
top_categoryz:'Restaurants','Food','Shopping','Beauty & Spas','Home Services','Health & Medical','Automotive','Nightlife','Local Services','Bars','Event Planning & Services','American (Traditional)','Active Life'
Tell me the category you're interested in:Restaurants
please enter day:Saturday
If will bring kids with you:Yes


23/12/05 22:17:57 WARN TaskSetManager: Stage 1 contains a task of very large size (3825 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+-----+------------+--------------------+
|                name|stars|review_count|             address|
+--------------------+-----+------------+--------------------+
|          Los Agaves|  4.5|        3834|     600 N Milpas St|
|          Mesa Verde|  4.5|        1796|       1919 Cliff Dr|
|    The Palace Grill|  4.5|        1500|         8 E Cota St|
|     Lure Fish House|  4.5|        1453|3815 State St, St...|
|       Lilly's Tacos|  4.5|        1219|      310 Chapala St|
|     Pickles & Swiss|  4.5|        1213|811 State St, unit E|
|    The Shop Kitchen|  4.5|        1108|     730 N Milpas St|
|Toma Restaurant &...|  4.5|        1084|324 West Cabrillo...|
| Finney's Crafthouse|  4.5|        1047|  35 State St, Ste A|
|  Cold Spring Tavern|  4.5|        1018|  5995 Stagecoach Rd|
|        Santo Mezcal|  4.5|         990|        119 State St|
|    East Beach Tacos|  4.5|         856|     226 S Milpas St|
|      Corazon Cocina|  4.5|         813|38 W Victoria,